# Import

In [2]:
# external libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import json
import torch
from datetime import datetime
from pathlib import Path

base_dir = "D:/Mario's files/SUTD/ISTD/Theory and Practice of Deep Learning/Project/project"

# change directory for loading files from disk and loading custom dictionaries
sys.path.append(base_dir)
os.chdir(base_dir)
print(os.getcwd())

D:\Mario's files\SUTD\ISTD\Theory and Practice of Deep Learning\Project\project


# Preprocess

In [3]:
sample_path = "./data/raw/trunc/Fri_Apr_2_2021/00_00_10.json"
raw_trunc_dir = "./data/raw/trunc/"
processed_dir = "./data/processed/"

Path(processed_dir).mkdir(parents=True, exist_ok=True)

In [4]:
# with open(sample_path) as json_file:
#     data = json.load(json_file)
# # data[0]

In [5]:
# assuming the sequence of each dataset is the same
def retrieve_traffics(PATH, columns=["SpeedBand"]):
    result = []
    with open(PATH) as json_file:
        data = json.load(json_file)
        for road in data:
            props = []
            for col in columns:
                p = road[col]
                props.append(p)
            result.append(props)
    return np.array(result)

# sort the directories by dates
def get_dates(dir_name):
    date_str_format = "%a_%b_%d_%Y"
    my_date = datetime.strptime(dir_name, date_str_format)
    return my_date

# sorted(os.listdir(raw_trunc_dir), key=get_dates)
def get_day_time(file_name):
    date_str_format = "%H_%M_%S"
    my_date = datetime.strptime(file_name, date_str_format)
    return my_date

def processed_get_day_time(file_path):
    head_tail = os.path.split(file_path)
    file_name = head_tail[-1].split(".")[0]
    return get_day_time(file_name)

def get_ordered_file_path(dir_path):
    sorted_dir_name = sorted(os.listdir(dir_path), key=get_dates)
    sorted_dir = [os.path.join(dir_path, d) for d in sorted_dir_name]

    file_path = []
    for d in sorted_dir:
        list_file = [os.path.join(d,f) for f in os.listdir(d)]
        sorted_list_file = sorted(list_file, key=processed_get_day_time)
        file_path.extend(sorted_list_file)
    return file_path

# test
print(retrieve_traffics(sample_path).shape) # (48.1)

(53, 1)


In [6]:
file_paths = get_ordered_file_path(raw_trunc_dir)

num_days = len(sorted(os.listdir(raw_trunc_dir), key=get_dates))
num_files = len(file_paths)
num_files_per_day = num_files/num_days

print("num_days = ", num_days)
print("num_files = ", num_files)
print("num_files_per_day = ", num_files_per_day)

num_days =  27
num_files =  7497
num_files_per_day =  277.6666666666667


In [7]:
sorted_dir_name = sorted(os.listdir(raw_trunc_dir), key=get_dates)
sorted_dir = [os.path.join(raw_trunc_dir, d) for d in sorted_dir_name]
for d in sorted_dir:
    print(f'In {d}, number of files = {len(os.listdir(d))}')

In ./data/raw/trunc/Mon_Mar_8_2021, number of files = 22
In ./data/raw/trunc/Tue_Mar_9_2021, number of files = 288
In ./data/raw/trunc/Wed_Mar_10_2021, number of files = 288
In ./data/raw/trunc/Thu_Mar_11_2021, number of files = 288
In ./data/raw/trunc/Fri_Mar_12_2021, number of files = 288
In ./data/raw/trunc/Sat_Mar_13_2021, number of files = 288
In ./data/raw/trunc/Sun_Mar_14_2021, number of files = 275
In ./data/raw/trunc/Mon_Mar_15_2021, number of files = 288
In ./data/raw/trunc/Tue_Mar_16_2021, number of files = 288
In ./data/raw/trunc/Wed_Mar_17_2021, number of files = 288
In ./data/raw/trunc/Thu_Mar_18_2021, number of files = 288
In ./data/raw/trunc/Fri_Mar_19_2021, number of files = 288
In ./data/raw/trunc/Sat_Mar_20_2021, number of files = 288
In ./data/raw/trunc/Sun_Mar_21_2021, number of files = 288
In ./data/raw/trunc/Mon_Mar_22_2021, number of files = 288
In ./data/raw/trunc/Tue_Mar_23_2021, number of files = 288
In ./data/raw/trunc/Wed_Mar_24_2021, number of files = 288


### Splitting

In [28]:
# Proposed data splitting
# make it a weekly basis
# if we do 70 20 10
# 7 weeks training data, 2 weeks validation, 1 week testing
# Reason being that there might be some periodicity in a weekly settings
# Splitting should not be random I think, they should be chronological


In [29]:
# sample of splitting (per minute)
training = 0.7
validation = 0.2
test = 0.1

file_paths = get_ordered_file_path(raw_trunc_dir)
size = len(file_paths)

train_partition = math.ceil(training*size)
val_partition = math.ceil(train_partition + validation*size)

train_paths, val_paths, test_paths = file_paths[:train_partition], file_paths[train_partition:val_partition], file_paths[val_partition:]

assert (len(train_paths)+len(val_paths)+len(test_paths)) == size, "Data size is not equal"

In [34]:
def get_processed_traffic(list_file_paths):
    """
    Takes list of file paths of traffic data in json format and retrieves the road features
    :param list_file_paths: file paths in a list format 
    :return:
        - Node features in a shape of (num_vertices, num_features,
    num_timesteps)
    """
    result = []
    for file_path in list_file_paths:
        result.append(retrieve_traffics(file_path))
    result = np.array(result) # (num_timesteps, num_vertices, num_features)
    return np.transpose(result, (1,2,0))

# save in processed
Path(processed_dir).mkdir(parents=True, exist_ok=True)

train_data, val_data, test_data = get_processed_traffic(train_paths), get_processed_traffic(val_paths), get_processed_traffic(test_paths)


In [35]:
print("train_data shape = ", train_data.shape)
print("val_data shape = ", val_data.shape)
print("test_data shape = ", test_data.shape)

train_data shape =  (53, 1, 5248)
val_data shape =  (53, 1, 1500)
test_data shape =  (53, 1, 749)


In [36]:
# save as npy
np.save(os.path.join(processed_dir, "train.npy"), train_data)
np.save(os.path.join(processed_dir, "val.npy"), val_data)
np.save(os.path.join(processed_dir, "test.npy"), test_data)

In [78]:
# def window_splitting(data ,window_size=6, output_size=1, step=1):
#     size = len(data)
#     x = []
#     y = []
#     for i in range(0, size-window_size-output_size+1, step):
#         x.append(data[i:i+window_size])
#         y.append(data[i+window_size:i+window_size+output_size])
#     return np.array(x), np.array(y)

# Load data

In [45]:
def generate_dataset(X, num_timesteps_input, num_timesteps_output):
    """
    Takes node features for the graph and divides them into multiple samples
    along the time-axis by sliding a window of size (num_timesteps_input+
    num_timesteps_output) across it in steps of 1.
    :param X: Node features of shape (num_vertices, num_features,
    num_timesteps)
    :return:
        - Node features divided into multiple samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_input).
        - Node targets for the samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_output).
    """
    # Generate the beginning index and the ending index of a sample, which
    # contains (num_points_for_training + num_points_for_predicting) points
    indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
               in range(X.shape[2] - (
                num_timesteps_input + num_timesteps_output) + 1)]

    # Save samples
    features, target = [], []
    for i, j in indices:
        features.append(
            X[:, :, i: i + num_timesteps_input].transpose(
                (0, 2, 1)))
        target.append(X[:, 0, i + num_timesteps_input: j])

    return torch.from_numpy(np.array(features)), \
           torch.from_numpy(np.array(target))

In [46]:
train_original_data = np.load(os.path.join(processed_dir, "train.npy"))
val_original_data = np.load(os.path.join(processed_dir, "val.npy"))
test_original_data = np.load(os.path.join(processed_dir, "test.npy"))

In [48]:
# similar to main.py
num_timesteps_input = 12
num_timesteps_output = 3

training_input, training_target = generate_dataset(train_original_data,
                                                   num_timesteps_input=num_timesteps_input,
                                                   num_timesteps_output=num_timesteps_output)
val_input, val_target = generate_dataset(val_original_data,
                                         num_timesteps_input=num_timesteps_input,
                                         num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

print(training_input.shape, training_target.shape)
print(val_input.shape, val_target.shape)
print(test_input.shape, test_target.shape)

torch.Size([5234, 53, 12, 1]) torch.Size([5234, 53, 3])
torch.Size([1486, 53, 12, 1]) torch.Size([1486, 53, 3])
torch.Size([735, 53, 12, 1]) torch.Size([735, 53, 3])


# Make function

In [3]:
# external libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import json
import torch
from copy import deepcopy
from datetime import datetime
from pathlib import Path

base_dir = "D:/Mario's files/SUTD/ISTD/Theory and Practice of Deep Learning/Project/project"

# change directory for loading files from disk and loading custom dictionaries
sys.path.append(base_dir)
os.chdir(base_dir)
print(os.getcwd())

D:\Mario's files\SUTD\ISTD\Theory and Practice of Deep Learning\Project\project


In [4]:
sample_path = "./data/raw/trunc/Fri_Apr_2_2021/00_00_10.json"
raw_trunc_dir = "./data/raw/trunc/"
processed_dir = "./data/processed/"

Path(processed_dir).mkdir(parents=True, exist_ok=True)

# assuming the sequence of each dataset is the same
def retrieve_traffics(PATH, columns=["SpeedBand"]):
    result = []
    with open(PATH) as json_file:
        data = json.load(json_file)
        for road in data:
            props = []
            for col in columns:
                p = road[col]
                props.append(p)
            result.append(props)
    return np.array(result)

# sort the directories by dates
def get_dates(dir_name):
    date_str_format = "%a_%b_%d_%Y"
    my_date = datetime.strptime(dir_name, date_str_format)
    return my_date

# sorted(os.listdir(raw_trunc_dir), key=get_dates)
def get_day_time(file_name):
    date_str_format = "%H_%M_%S"
    my_date = datetime.strptime(file_name, date_str_format)
    return my_date

def processed_get_day_time(file_path):
    head_tail = os.path.split(file_path)
    file_name = head_tail[-1].split(".")[0]
    return get_day_time(file_name)

def get_ordered_file_path(dir_path):
    sorted_dir_name = sorted(os.listdir(dir_path), key=get_dates)
    sorted_dir = [os.path.join(dir_path, d) for d in sorted_dir_name]

    file_path = []
    for d in sorted_dir:
        list_file = [os.path.join(d,f) for f in os.listdir(d)]
        sorted_list_file = sorted(list_file, key=processed_get_day_time)
        file_path.extend(sorted_list_file)
    return file_path

In [5]:
def get_adjacency(file_path):
    '''
    Generates an Adjacency matrix
    -----------------------------
    :param str file_path: the file path of the dataset
    -----------------------------
    :returns:
        npy: Adjacency matrix
    '''
    with open(file_path, 'r') as traffic_data_file:
        traffic_records = json.load(traffic_data_file)
    
    # Format traffic_records so we can get the start & end location positions of the link
    traffic_records_formatted = []
    for record in traffic_records:
        record = deepcopy(record)
        lat_long_positions = record['Location'].split()
        record['start_pos'] = ' '. join(lat_long_positions[0:2])
        record['end_pos'] = ' '. join(lat_long_positions[2:4])
        traffic_records_formatted.append(record)
    traffic_records_formatted.sort(key=lambda x: int(x.get('LinkID')))
    
    # Generate Node Mappings
    nodes_params_dict = {}
    Nodes2LinkID = {} # not needed
    LinkID2Nodes = {} # not needed
    for i, record in enumerate(traffic_records_formatted):
        record = deepcopy(record)
        Nodes2LinkID[i] = record['LinkID']
        LinkID2Nodes[record['LinkID']] = i
        nodes_params_dict[i] = record
    
    
    # Generating a Directed Adjacency matrix
    '''
    Refer to illustrations
    To find a directed adjacency, we need to check each link(node)
    There is a directed adjacency from Node A to Node B if the end_pos of Node A is the start_pos of Node B

    This involves us having to loop through all nodes with: O(n^2) complexity
    (Computation and speed optimisation is not a concern here as this is pre-generated before training)
    '''
    nodes_count = len(nodes_params_dict)
    A = np.zeros((nodes_count,nodes_count), dtype=int)  
    # Finding the directed edges of the nodes
    for i, i_record in nodes_params_dict.items():
        # print(f'=====> Finding edges for Node: {i}, LinkID: {i_record["LinkID"]}')
        for j, j_record in nodes_params_dict.items():
            if i_record['end_pos'] == j_record['start_pos']:
                # print(f'Found a Directed Edge from Node {i} to Node {j}')
                A[i,j] = 1
    return A

def link_length(start_pos, end_pos):
    """
    Calculation of distance between two lat-long geo positions, using Haversine distance
    ------------------------------------
    :param string start_pos: lat & long separated with a space
    :param string end_pos: lat & long separated with a space
    ------------------------------------
    :returns:
        float: total length of the link
    """
    lat1, lon1 = [float(pos) for pos in start_pos.split()]
    lat2, lon2 = [float(pos) for pos in end_pos.split()]
    radius = 6371
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    d = radius * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return d


def get_features(file_path):
    '''
    Generates a Feature matrix
    Note: Feature Matrix, X, would contain the output speedband as well. 
    -----------------------------
    :param str file_path: the file path of the dataset
    -----------------------------
    :returns:
        npy: Feature matrix
        dict: Metadata of parameters
    '''
    with open(file_path, 'r') as traffic_data_file:
        traffic_records = json.load(traffic_data_file)
    
    # Find out all Road Categories
    roadcategory_list = []
    for record in traffic_records:
        if record['RoadCategory'] not in roadcategory_list:
            roadcategory_list.append(record['RoadCategory'])
    roadcategory_list.sort()
    Index2RoadCat = {}
    RoadCat2Index = {}
    for i, cat in enumerate(roadcategory_list):
        Index2RoadCat[i] = cat
        RoadCat2Index[cat] = i

    
    # Format traffic_records to include additional field on length of link
    traffic_records_formatted = []
    for record in traffic_records:
        record = deepcopy(record)
        lat_long_positions = record['Location'].split()
        record['start_pos'] = ' '. join(lat_long_positions[0:2])
        record['end_pos'] = ' '. join(lat_long_positions[2:4])
        record['length'] = link_length(record['start_pos'], record['end_pos'])
        traffic_records_formatted.append(record)
    
    # Generate Node Mappings
    nodes_params_dict = {}
    Nodes2LinkID = {} # not needed
    LinkID2Nodes = {} # not needed
    for i, record in enumerate(traffic_records_formatted):
        record = deepcopy(record)
        Nodes2LinkID[i] = record['LinkID']
        LinkID2Nodes[record['LinkID']] = i
        nodes_params_dict[i] = record
        
    nodes_count = len(nodes_params_dict)
    num_features = 3
    X = []
    # Positions of Features
    # 0. SpeedBand
    # 1. RoadCategory
    # 2. Length of Link
    for i, record in nodes_params_dict.items():
        features = [float(record['SpeedBand']),RoadCat2Index[record['RoadCategory']],record['length'] ]
        X.append(features)
#         X[i][0] = float(record['SpeedBand'])
#         X[i][1] = RoadCat2Index[record['RoadCategory']]
#         X[i][2] = record['length']
    
    
    return np.array(X), nodes_params_dict, RoadCat2Index

In [23]:
def processed(files_dir, process_dir, overwrite=False):
    '''
    Process traffic data in json file and save them in numpy array
    If overwrite=False, do not process the data if the processed files already exist
    -----------------------------
    :param str files_dir: the directory of the raw dataset
           str files_dir: the directory of the processed output
    -----------------------------
    :returns: None
    '''
    Path(process_dir).mkdir(parents=True, exist_ok=True)
    
    # check if files are already processed
    dataset_path = os.path.join(process_dir, "dataset.npy")
    adj_path = os.path.join(process_dir, "adj.npy")
    metadata_path = os.path.join(process_dir, "metadata.json")
    
    if (not overwrite and
            (os.path.isfile(dataset_path)
            and os.path.isfile(adj_path)
            and os.path.isfile(metadata_path))):
        # do not run the function if both overwrite is false and all processed files already exist
        return
    
    file_paths = get_ordered_file_path(files_dir)
    
    X = []
    road_cat2index = {}
    
    for data_file_path in file_paths:
        print(f"Processing {data_file_path}")
        features,_,RoadCat2Index = get_features(data_file_path)
        
        for k in RoadCat2Index.keys():
            if k not in road_cat2index:
                road_cat2index[k] = RoadCat2Index[k]
        X.append(features)
        
    X = np.transpose(X, (1,2,0)) # (num_vertices, num_features, num_timesteps)
    A = get_adjacency(file_paths[0])

    # save both
    np.save(dataset_path, X)
    np.save(adj_path, A)
    
    # save metadata
    with open(metadata_path, 'w') as outfile:
        json.dump(road_cat2index, outfile)
    
    print("Done")

def load(process_dir):
    '''
    Load datasets and adjacency matrixs from numpy file. Also calculates the means and stds
    -----------------------------
    :param str process_dir:  the directory of the processed output
    -----------------------------
    :returns: 
        npy: Adjacency matrix
        npy: Feature matrix
        dict: Metadata
        npy: means 
        npy: stds
    '''
    dataset_path = os.path.join(process_dir, "dataset.npy")
    adj_path = os.path.join(process_dir, "adj.npy")
    metadata_path = os.path.join(process_dir, "metadata.json")

    A = np.load(adj_path)
    X = np.load(dataset_path)
    X = X.astype(np.float32)

    with open(metadata_path) as json_file:
        metadata = json.load(json_file)
        
    # Normalization using Z-score method
    means = np.mean(X, axis=(0, 2))
    X = X - means.reshape(1, -1, 1)
    stds = np.std(X, axis=(0, 2))
    X = X / stds.reshape(1, -1, 1)

    return A, X, metadata, means, stds

def denormalize(X, stds, means):
    return X * stds.reshape(1, -1, 1) + means.reshape(1, -1, 1)

def get_normalized_adj(A):
    """
    Returns the degree normalized adjacency matrix.
    """
    A = A + np.diag(np.ones(A.shape[0], dtype=np.float32))
    D = np.array(np.sum(A, axis=1)).reshape((-1,))
    D[D <= 10e-5] = 10e-5    # Prevent infs
    diag = np.reciprocal(np.sqrt(D))
    A_wave = np.multiply(np.multiply(diag.reshape((-1, 1)), A),
                         diag.reshape((1, -1)))
    return A_wave

def generate_dataset(X, num_timesteps_input, num_timesteps_output):
    """
    Takes node features for the graph and divides them into multiple samples
    along the time-axis by sliding a window of size (num_timesteps_input+
    num_timesteps_output) across it in steps of 1.
    :param X: Node features of shape (num_vertices, num_features,
    num_timesteps)
    :return:
        - Node features divided into multiple samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_input).
        - Node targets for the samples. Shape is
          (num_samples, num_vertices, num_features, num_timesteps_output).
    """
    # Generate the beginning index and the ending index of a sample, which
    # contains (num_points_for_training + num_points_for_predicting) points
    indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
               in range(X.shape[2] - (
                num_timesteps_input + num_timesteps_output) + 1)]

    # Save samples
    features, target = [], []
    for i, j in indices:
        features.append(
            X[:, :, i: i + num_timesteps_input].transpose(
                (0, 2, 1)))
        target.append(X[:, 0, i + num_timesteps_input: j])

    return torch.from_numpy(np.array(features)), \
           torch.from_numpy(np.array(target))

In [24]:
file_paths = get_ordered_file_path(raw_trunc_dir)

In [25]:
sample_path = "./data/raw/trunc/Fri_Apr_2_2021/00_00_10.json"
raw_trunc_dir = "./data/raw/trunc/"
process_dir = "./data/processed/"

Path(process_dir).mkdir(parents=True, exist_ok=True)

In [26]:
processed(raw_trunc_dir, process_dir, overwrite=False)

Processing ./data/raw/trunc/Mon_Mar_8_2021\22_10_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_15_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_20_10.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_25_07.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_30_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_35_20.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_40_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_45_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_50_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\22_55_12.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_00_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_05_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_10_09.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_15_08.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_20_10.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_25_10.json
Processing ./data/raw/trunc/Mon_Mar_8_2021\23_30_09.json
Processing ./data/raw/trunc/Mon

Processing ./data/raw/trunc/Tue_Mar_9_2021\14_30_08.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\14_35_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\14_40_10.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\14_45_12.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\14_50_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\14_55_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_00_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_05_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_10_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_15_20.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_20_12.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_25_12.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_30_08.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_35_10.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_40_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_45_09.json
Processing ./data/raw/trunc/Tue_Mar_9_2021\15_50_08.json
Processing ./data/raw/trunc/Tue

Processing ./data/raw/trunc/Wed_Mar_10_2021\04_25_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_30_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_35_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_40_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_45_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_50_18.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\04_55_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_00_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_05_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_10_20.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_15_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_20_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_25_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_30_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_35_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_40_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\05_45_08.json
Processing ./d

Processing ./data/raw/trunc/Wed_Mar_10_2021\19_20_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_25_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_30_10.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_35_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_40_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_45_21.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_50_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\19_55_10.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_00_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_05_14.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_10_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_15_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_20_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_25_08.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_30_10.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_35_09.json
Processing ./data/raw/trunc/Wed_Mar_10_2021\20_40_08.json
Processing ./d

Processing ./data/raw/trunc/Thu_Mar_11_2021\10_20_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_25_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_30_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_35_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_40_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_45_10.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_50_20.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\10_55_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_00_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_05_10.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_10_14.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_15_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_20_08.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_25_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_30_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_35_09.json
Processing ./data/raw/trunc/Thu_Mar_11_2021\11_40_08.json
Processing ./d

...

Processing ./data/raw/trunc/Thu_Apr_1_2021\07_30_10.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\07_35_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\07_40_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\07_45_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\07_50_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\07_55_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_00_21.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_05_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_10_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_15_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_20_21.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_25_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_30_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_35_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_40_16.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_45_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\08_50_09.json
Processing ./data/raw/trunc/Thu

Processing ./data/raw/trunc/Thu_Apr_1_2021\22_35_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\22_40_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\22_45_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\22_50_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\22_55_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_00_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_05_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_10_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_15_17.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_20_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_25_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_30_11.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_35_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_40_09.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_45_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_50_08.json
Processing ./data/raw/trunc/Thu_Apr_1_2021\23_55_08.json
Processing ./data/raw/trunc/Fri

Processing ./data/raw/trunc/Fri_Apr_2_2021\13_35_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\13_40_10.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\13_45_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\13_50_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\13_55_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_00_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_05_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_10_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_15_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_20_11.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_25_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_30_20.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_35_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_40_08.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_45_09.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_50_17.json
Processing ./data/raw/trunc/Fri_Apr_2_2021\14_55_09.json
Processing ./data/raw/trunc/Fri

Processing ./data/raw/trunc/Sat_Apr_3_2021\05_00_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_05_17.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_10_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_15_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_20_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_25_18.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_30_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_35_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_40_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_45_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_50_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\05_55_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\06_00_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\06_05_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\06_10_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\06_15_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\06_20_09.json
Processing ./data/raw/trunc/Sat

Processing ./data/raw/trunc/Sat_Apr_3_2021\20_00_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_05_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_10_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_15_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_20_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_25_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_30_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_35_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_40_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_45_10.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_50_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\20_55_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\21_00_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\21_05_19.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\21_10_09.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\21_15_08.json
Processing ./data/raw/trunc/Sat_Apr_3_2021\21_20_11.json
Processing ./data/raw/trunc/Sat

In [28]:
A, X, metadata, means, stds = load(process_dir)
print(A.shape)
print(X.shape)
print(metadata)
print(means)
print(stds)

(53, 53)
(53, 3, 7497)
{'B': 0, 'C': 1}
[3.993683   0.9056604  0.05673373]
[1.0052049  0.29230058 0.03545753]


In [33]:
num_timesteps_input = 6 # 30 minutes
num_timesteps_output = 2 # 10 minutes

split_line1 = int(X.shape[2] * 0.6)
split_line2 = int(X.shape[2] * 0.8)

train_original_data = X[:, :, :split_line1]
val_original_data = X[:, :, split_line1:split_line2]
test_original_data = X[:, :, split_line2:]

training_input, training_target = generate_dataset(train_original_data,
                                                   num_timesteps_input=num_timesteps_input,
                                                   num_timesteps_output=num_timesteps_output)
val_input, val_target = generate_dataset(val_original_data,
                                         num_timesteps_input=num_timesteps_input,
                                         num_timesteps_output=num_timesteps_output)
test_input, test_target = generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

A_wave = get_normalized_adj(A)
A_wave = torch.from_numpy(A_wave)

In [34]:
training_input.shape

torch.Size([4491, 53, 6, 3])

In [35]:
training_target.shape

torch.Size([4491, 53, 2])

In [37]:
denormalize(training_target, stds[0], means[0])

tensor([[[4., 4.],
         [5., 4.],
         [4., 4.],
         ...,
         [4., 5.],
         [4., 4.],
         [4., 5.]],

        [[4., 4.],
         [4., 4.],
         [4., 5.],
         ...,
         [5., 5.],
         [4., 4.],
         [5., 5.]],

        [[4., 4.],
         [4., 4.],
         [5., 5.],
         ...,
         [5., 4.],
         [4., 4.],
         [5., 4.]],

        ...,

        [[3., 3.],
         [3., 3.],
         [4., 4.],
         ...,
         [4., 4.],
         [3., 3.],
         [5., 5.]],

        [[3., 3.],
         [3., 3.],
         [4., 4.],
         ...,
         [4., 4.],
         [3., 3.],
         [5., 5.]],

        [[3., 3.],
         [3., 3.],
         [4., 6.],
         ...,
         [4., 4.],
         [3., 3.],
         [5., 5.]]])